# ⬇️ Instalación y carga del modelo

In [ ]:
%%capture
!pip install -U transformers
!pip install accelerate
!pip install -U bitsandbytes
!pip install Wikipedia-API

In [ ]:
import wikipediaapi

# Initializate Wikipedia object
wikipedia = wikipediaapi.Wikipedia(user_agent='QuestionGeneration (francoartico0@gmail.com)', language='es')

def clean_wikipedia_article(title, sections_to_remove):
    """
    Fetches a Wikipedia article and removes specified sections
    """
    page = wikipedia.page(title)

    if page.exists():
      for section in sections_to_remove:
          section_to_remove = page.section_by_title(section)
          if section_to_remove:
            page.sections.remove(section_to_remove)
      return page.text
    return None


In [ ]:
sections_to_remove =["Enlaces externos", "Bibliografía", "Referencias", "Notas", "Véase también", "Notas y referencias", "Otras lecturas"]
article = clean_wikipedia_article(title="Bill Gates", sections_to_remove=sections_to_remove)

print(article)

William Henry Gates III (Seattle, 28 de octubre de 1955), conocido como Bill Gates, es un magnate empresarial, desarrollador de software, inversor, autor y filántropo estadounidense. Es cofundador de Microsoft, junto con su difunto amigo de la infancia Paul Allen. Durante su carrera en Microsoft, Gates ocupó los cargos de presidente, director ejecutivo (CEO), presidente y arquitecto jefe de software, además de ser el mayor accionista individual hasta mayo de 2014. Fue uno de los principales empresarios de la revolución de las microcomputadoras de las décadas de 1970 y 1980.
Su fortuna se calcula en 129 000 millones de dólares (2023) según la revista Forbes, hecho que lo colocó como el cuarto hombre más rico del mundo.[3]​ Antes del estallido de la burbuja de las punto com, su patrimonio neto ascendió a 114 100 millones de dólares, lo que lo convirtió en la décima persona más rica en toda la historia de la humanidad.[4]​
Es uno de los empresarios más conocidos que surgieron durante los 

In [ ]:
def generate_response(model, messages, tokenizer):
    # Generate inputs using chat template
    inputs = tokenizer.apply_chat_template(messages,
                                           tokenize=True,
                                           add_generation_prompt=True,
                                           return_tensors="pt").to("cuda")

    print(f"Number of inputs tokens: {len(inputs[0])}")

    # Create a TextStreamer for real-time output
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)

    # Generte text
    _ = model.generate(pad_token_id=tokenizer.pad_token_id,
                       input_ids=inputs,
                       streamer=text_streamer,
                       max_new_tokens=1024,
                       use_cache=True)

In [ ]:
import torch
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer

access_token = userdata.get('HF_TOKEN')

checkpoint = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                          token=access_token,
                                          return_tensors="pt"
)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             device_map="auto",
                                             token=access_token,
                                             torch_dtype=torch.float16,
                                             quantization_config=nf4_config

)

# Ensure tokenizer had pad token
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

# ✅ Generación de preguntas con `Llama-3.1-8B-Instruct` usando zero-shot

In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "Eres un generador de preguntas de lectura comprensiva. "
            "Tu tarea es crear preguntas basadas en un texto dado, "
            "sin incluir respuestas ni opciones. Solo debes generar preguntas, "
            "en las cuales deberás incluir preguntas de tipo verdadero o falso."
        ),
    },
    {
        "role": "user",
        "content": (
            "Genera exactamente 10 preguntas, de las cuales 5 deben ser de tipo "
            "verdadero o falso, sin incluir las respuestas ni opciones, sobre el siguiente texto:\n\n"
            f"{article}"
        ),
    },
]

generate_response(model, messages, tokenizer)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Number of inputs tokens: 3062
Aquí te presento 10 preguntas basadas en el texto, 5 de ellas son de tipo verdadero o falso:

1. ¿Cuál fue el cargo que ocupó Bill Gates en Microsoft antes de dejar la empresa en 2008?
2. ¿En qué año se casó Bill Gates con Melinda French?
3. (Verdadero o falso) Bill Gates nació en el seno de una familia católica.
4. ¿Cuál fue el nombre del manuscrito de Leonardo da Vinci que adquirió Bill Gates en 1994?
5. (Verdadero o falso) Bill Gates es el fundador de la empresa PayPal.

6. ¿Cuál fue el cargo que ocupó Bill Gates en la Fundación Bill y Melinda Gates después de dejar Microsoft?
7. ¿En qué año se anunció la donación de 20 000 millones de dólares de Bill Gates a la Fundación Bill y Melinda Gates?
8. (Verdadero o falso) Bill Gates es el creador del sistema operativo Windows.
9. ¿Cuál fue el nombre del partido de tenis no competitivo que jugó Bill Gates en 2017?
10. ¿Cuántas empresas forman parte de la propiedad de Bill Gates?<|eot_id|>


# 💥 Generación de preguntas con `Llama-3.1-8B-Instruct` usando few-shot

In [ ]:
# Clean Wikipedia articles
turing = clean_wikipedia_article(title="Alan Turing", sections_to_remove=sections_to_remove)
von_neumann = clean_wikipedia_article(title="John von Neumann", sections_to_remove=sections_to_remove)

# Example questions for Alan Turing
turing_question_examples = """
Preguntas de tipo respuestas cortas:
1. ¿En qué fecha nació Alan Turing?
2. ¿Dónde nació Alan Turing?
3. ¿Cuáles eran las profesiones de Alan Turing?
4. ¿Cuál fue la causa de su muerte?
5. ¿A qué se dedicó durante la Segunda Guerra Mundial?

Preguntas de tipo Verdadero o Falso:
6. La máquina sobre la cual trabajó durante la guerra se llamó máquina Enigma. ¿Verdadero o Falso?
7. Se estima que el trabajo de Turing durante la guerra acortó la misma entre 2 y 4 años. ¿Verdadero o Falso?
8. La prueba de Turing se utiliza para juzgar la computabilidad de una función. ¿Verdadero o Falso?
9. Alan Turing fue condenado por crímenes de guerra después de la Segunda Guerra Mundial. ¿Verdadero o Falso?
"""

# Example questions for John von Neumann
von_neumann_question_examples = """
Preguntas de tipo respuestas cortas:
1. ¿A qué se dedicaba John von Neumann?
2. ¿Cuál fue la fecha de su nacimiento?
3. ¿En qué universidad obtuvo su doctorado en matemáticas?
4. ¿Cuál fue su aporte en la Segunda Guerra Mundial?

Preguntas de tipo Verdadero o Falso:
5. Es uno de los matemáticos más importantes del siglo XX. ¿Verdadero o Falso?
6. Asistió a la Universidad de Budapest. ¿Verdadero o Falso?
7. John von Neumann participó en el famoso Proyecto Manhattan. ¿Verdadero o Falso?
"""

# Define messages for the model
messages = [
    {
        "role": "system",
        "content": (
            "Eres un generador de preguntas de lectura comprensiva. "
            "Tu tarea es crear preguntas basadas en un texto dado, "
            "sin incluir respuestas ni opciones. Solo debes generar preguntas, "
            "incluyendo preguntas de tipo verdadero o falso."
        ),
    },
    {
        "role": "user",
        "content": (
            f"Usando el siguiente artículo {turing}, estos son ejemplos de preguntas que puedes hacer:\n"
            f"{turing_question_examples}\n\n"
            f"Usando el siguiente artículo {von_neumann}, estos son ejemplos de preguntas que puedes hacer:\n"
            f"{von_neumann_question_examples}\n\n"
            f"Genera exactamente 10 preguntas, incluyendo 5 que deben ser de tipo verdadero o falso, sobre el siguiente texto:\n{article}"
        ),
    },
]

# Generate response
generate_response(model, messages, tokenizer)

Number of inputs tokens: 19763
Preguntas de tipo respuestas cortas:

1. ¿Quién es Bill Gates?
2. ¿Cuál fue el año de nacimiento de Bill Gates?
3. ¿Qué empresa fundó Bill Gates junto con Paul Allen?
4. ¿Cuál es el nombre de la fundación de Bill Gates y su esposa Melinda?
5. ¿Cuál es el nombre del evento deportivo de caridad en el que participó Bill Gates?

Preguntas de tipo Verdadero o Falso:

6. Bill Gates es el fundador de la empresa Apple. ¿Verdadero o Falso?
7. Bill Gates es un gran aficionado al tenis. ¿Verdadero o Falso?
8. Bill Gates se casó con Melinda French en 1993. ¿Verdadero o Falso?
9. Bill Gates tiene tres hijos. ¿Verdadero o Falso?
10. Bill Gates donó 20 millones de dólares a la Universidad de Harvard. ¿Verdadero o Falso?<|eot_id|>


# ✨ Generación de preguntas con el modelo ajustado

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
from google.colab import userdata
import torch

access_token = userdata.get('HF_TOKEN')

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

messages = [
    {
        "role": "system",
        "content": (
            "Eres un generador de preguntas de lectura comprensiva. "
            "Tu tarea es crear preguntas basadas en un texto dado, "
            "sin incluir respuestas ni opciones. Solo debes generar preguntas, "
            "incluyendo preguntas de tipo verdadero o falso."
        ),
    },
    {
        "role": "user",
        "content": (
            f"Genera exactamente 10 preguntas, sobre el siguiente texto:\n{article}"
        ),
    },
]

tokenizer = AutoTokenizer.from_pretrained("Doberfran/spanish-question-generator",
                                          token=access_token,
                                          return_tensor="pt")
model = AutoModelForCausalLM.from_pretrained("Doberfran/spanish-question-generator",
                                             device_map="auto",
                                             token=access_token,
                                             torch_dtype=torch.float16,
                                             quantization_config=nf4_config
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
generate_response(model, messages, tokenizer)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Number of inputs tokens: 3034
1. ¿En qué año nació Bill Gates? 
2. ¿Cuál es la ocupación actual de Bill Gates en Microsoft después de dejar el cargo de CEO en 2008?
3. ¿Cuál fue la ocupación de Bill Gates en Microsoft durante su carrera en la empresa?
4. ¿Con quién se casó Bill Gates en 1994?
5. ¿Cuál es la fortuna estimada de Bill Gates según la revista Forbes en 2023?
6. ¿Con quién firmó un acuerdo de donaciones en 1999?
7. ¿Cuál es la organización con la que Bill Gates y su esposa, Melinda, trabajan juntos?
8. ¿Cuándo dejó Bill Gates de dirigir Microsoft?
9. ¿Cuál es la segunda ocupación de Bill Gates desde 2007?
10. ¿Cuál es el honor que recibió Bill Gates en 2005 de parte del Reino Unido?<|eot_id|>
